In [ ]:
# import packages
import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from matplotlib import pyplot as plt

In [20]:
# a function to extract the sources I'll be analyzing. I'll focus on the english ones
def getSources():
    source_url = 'https://newsapi.org/v1/sources?language=en'
    response = requests.get(source_url).json()
    sources = []
    for source in response['sources']:
        sources.append(source['id'])
    return sources

In [21]:
# let's see what news sources we have
sources = getSources()
print('number of sources :', len(sources))
print('sources :', sources)

('number of sources :', 60)
('sources :', [u'abc-news-au', u'al-jazeera-english', u'ars-technica', u'associated-press', u'bbc-news', u'bbc-sport', u'bloomberg', u'breitbart-news', u'business-insider', u'business-insider-uk', u'buzzfeed', u'cnbc', u'cnn', u'daily-mail', u'engadget', u'entertainment-weekly', u'espn', u'espn-cric-info', u'financial-times', u'football-italia', u'fortune', u'four-four-two', u'fox-sports', u'google-news', u'hacker-news', u'ign', u'independent', u'mashable', u'metro', u'mirror', u'mtv-news', u'mtv-news-uk', u'national-geographic', u'new-scientist', u'newsweek', u'new-york-magazine', u'nfl-news', u'polygon', u'recode', u'reddit-r-all', u'reuters', u'talksport', u'techcrunch', u'techradar', u'the-economist', u'the-guardian-au', u'the-guardian-uk', u'the-hindu', u'the-huffington-post', u'the-lad-bible', u'the-new-york-times', u'the-next-web', u'the-sport-bible', u'the-telegraph', u'the-times-of-india', u'the-verge', u'the-wall-street-journal', u'the-washington-p

In [22]:
def mapping():
    d={}
    response=requests.get("https://newsapi.org/v1/sources?language=en")
    response=response.json()
    for s in response['sources']:
        d[s['id']]=s['category']
    return d

In [23]:
m=mapping()
print('category of reuters:', m['reuters'])
print('category of techcrunch:', m['techcrunch'])



('category of reuters:', u'general')
('category of techcrunch:', u'technology')


In [24]:
print('categories:', list(set(m.values())))

('categories:', [u'gaming', u'business', u'entertainment', u'general', u'science-and-nature', u'music', u'politics', u'sport', u'technology'])


In [25]:
def category(source,m):
    try:
        return m[source]
    except:
        return 'NC'

In [26]:
def cleanData(path):
    data=pd.read_csv(path)
    data=data.drop_duplicates('url')
    data.to_csv(path,index=False)

In [29]:
def getDailyNews():
    sources = getSources()
    key = 'de90302e18eb48c19dec162d8fe900a3'
    url = 'https://newsapi.org/v1/articles?source={0}&sortBy={1}&apiKey={2}'
    responses = []
    for i, source in tqdm(enumerate(sources)):
        try:
            u = url.format(source, 'top',key)
            response = requests.get(u)
            r = response.json()
            for article in r['articles']:
                article['source'] = source
            responses.append(r)
        except:
            u = url.format(source, 'latest', key)
            response = requests.get(u)
            r = response.json()
            for article in r['articles']:
                article['source'] = source
            responses.append(r)
      
    news = pd.DataFrame(reduce(lambda x,y: x+y ,map(lambda r: r['articles'], responses)))
    news = news.dropna()
    news = news.drop_duplicates()
    d = mapping()
    news['category'] = news['source'].map(lambda s: category(s, d))
    news['scraping_date'] = datetime.now()

    try:
        aux = pd.read_csv('news.csv')
    except:
        aux = pd.DataFrame(columns=list(news.columns))
        aux.to_csv('news.csv', encoding='utf-8', index=False)

    with open('news.csv', 'a') as f:
        news.to_csv(f, header=False, encoding='utf-8', index=False)

    cleanData('news.csv')
    print('Done')
print("OK")

OK


In [30]:
if __name__ == '__main__':
    getDailyNews()


60it [03:02,  2.97s/it]


Done
